In [1]:
!pip install transformers datasets seqeval torch accelerate

     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     --------- ------------------------------ 10.2/43.6 kB ? eta -:--:--
     -------------------------- ----------- 30.7/43.6 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 43.6/43.6 kB 354.9 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ----------------------------------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [78]:
import json
from datasets import Dataset
with open("datasets/intent_ner_dataset.json",
            "r",
            encoding="utf-8") as f:
    data = json.load(f)

with open("datasets/output.json",
            "r",
            encoding="utf-8") as f:
    ddata = json.load(f)

count = 0
for i in ddata:
    if len(i["tokens"]) != len(i["ner_tags"]):
        count += 1
        print(i)
print(len(ddata), count)

289 0


In [48]:
fix = [
    {"tokens": ["Retrieve", "all", "drone", "intrusion", "alerts", "on", "March", "5,", "2024", "."], 
     "ner_tags": ["B-ACTION", "I-ACTION", "B-ALERT_TYPE", "I-ALERT_TYPE", "O", "B-DATE", "I-DATE", "I-DATE", "O"]},

    {"tokens": ["How", "many", "red", "alerts", "where", "there", "in", "Dead", "Sea", "this", "month", "?"], 
     "ner_tags": ["B-QUESTION_TYPE", "I-QUESTION_TYPE", "B-ALERT_TYPE", "I-ALERT_TYPE", "O", "B-LOCATION", "I-LOCATION", "B-TIME_FRAME", "I-TIME_FRAME", "O"]},

    {"tokens": ["How", "many", "red", "alerts", "where", "there", "in", "Ein", "Kamonim", "last", "week", "?"], 
     "ner_tags": ["B-QUESTION_TYPE", "I-QUESTION_TYPE", "B-ALERT_TYPE", "I-ALERT_TYPE", "O", "O", "O", "B-LOCATION", "B-TIME_FRAME", "I-TIME_FRAME", "O"]},

    {"tokens": ["Retrieve", "all", "red", "alerts", "on", "March", "5,", "2024", "."], 
     "ner_tags": ["B-ACTION", "I-ACTION", "O", "B-ALERT_TYPE", "I-ALERT_TYPE", "O", "B-DATE", "I-DATE", "I-DATE", "O"]},

    {"tokens": ["Has", "there", "been", "any", "drone", "intrusion", "alerts", "on", "yesterday", "?"], 
     "ner_tags": ["B-QUESTION_TYPE", "I-QUESTION_TYPE", "O", "O", "B-ALERT_TYPE", "I-ALERT_TYPE", "O", "B-DATE", "O"]},

    {"tokens": ["How", "many", "missile", "alerts", "where", "there", "in", "Southern", "Negev", "this", "month", "?"], 
     "ner_tags": ["B-QUESTION_TYPE", "I-QUESTION_TYPE", "B-ALERT_TYPE", "I-ALERT_TYPE", "O", "O", "B-LOCATION", "I-LOCATION", "B-TIME_FRAME", "I-TIME_FRAME", "O"]},

    {"tokens": ["Give", "me", "all", "red", "alerts", "on", "April", "15,", "2023."], 
     "ner_tags": ["B-ACTION", "I-ACTION", "O", "B-ALERT_TYPE", "I-ALERT_TYPE", "O", "B-DATE", "I-DATE", "I-DATE", "O"]},

    {"tokens": ["Tell", "me", "the", "number", "of", "drone", "intrusion", "alerts", "on", "January", "1,", "2022."], 
     "ner_tags": ["B-ACTION", "O", "B-COMP_OP", "O", "B-ALERT_TYPE", "I-ALERT_TYPE", "O", "B-DATE", "I-DATE", "I-DATE", "O"]},

    {"tokens": ["Give", "me", "all", "rocket", "alerts", "on", "January", "1,", "2022."], 
     "ner_tags": ["B-ACTION", "I-ACTION", "O", "B-ALERT_TYPE", "I-ALERT_TYPE", "O", "B-DATE", "I-DATE", "I-DATE", "O"]},

    {"tokens": ["How", "many", "red", "alerts", "where", "there", "in", "Arabah", "this", "month", "?"], 
     "ner_tags": ["B-QUESTION_TYPE", "I-QUESTION_TYPE", "B-ALERT_TYPE", "I-ALERT_TYPE", "O", "B-LOCATION", "B-TIME_FRAME", "I-TIME_FRAME", "O"]},

    {"tokens": ["Retrieve", "all", "red", "alerts", "on", "March", "5,", "2024."], 
     "ner_tags": ["B-ACTION", "I-ACTION", "O", "B-ALERT_TYPE", "I-ALERT_TYPE", "O", "B-DATE", "I-DATE", "I-DATE", "O"]},

    {"tokens": ["How", "many", "red", "alerts", "where", "there", "in", "Upper", "Galilee", "?"], 
     "ner_tags": ["B-QUESTION_TYPE", "I-QUESTION_TYPE", "B-ALERT_TYPE", "I-ALERT_TYPE", "O", "B-LOCATION", "I-LOCATION", "O"]},

    {"tokens": ["Has", "there", "been", "any", "rocket", "alerts", "on", "yesterday", "?"], 
     "ner_tags": ["B-QUESTION_TYPE", "I-QUESTION_TYPE", "O", "O", "B-ALERT_TYPE", "I-ALERT_TYPE", "O", "B-DATE", "O"]},

    {"tokens": ["How", "many", "missile", "alerts", "where", "there", "in", "Lakhish", "yesterday", "?"], 
     "ner_tags": ["B-QUESTION_TYPE", "I-QUESTION_TYPE", "B-ALERT_TYPE", "I-ALERT_TYPE", "O", "B-LOCATION", "B-TIME_FRAME", "I-TIME_FRAME", "O"]}
]

fix_count = 0
for f in fix:
    flag = fix_count
    for d in ddata:
        if " ".join(f["tokens"]) == " ".join(d["tokens"]):
            d["ner_tags"] = f["ner_tags"]
            fix_count += 1
            break
    if flag == fix_count:
        print(f["tokens"], d["tokens"])
print(fix_count, len(fix))
with open("datasets/output.json",
            "w",
            encoding="utf-8") as f:
    json.dump(ddata, f, ensure_ascii=False, indent=4)

14 14


In [22]:


unique_tags = list(set(tag for sample in data for tag in sample["ner_tags"]))
print(unique_tags) 

# Create label mapping
label2id = {label: i for i, label in enumerate(unique_tags)}
id2label = {i: label for label, i in label2id.items()}
print(label2id)

intent2id = {"COUNT_ALERTS": 0, "CHECK_ALERTS": 1, "SHOW_ALERTS": 2, "TIME_ALERTS":3}
id2intent = {v: k for k, v in intent2id.items()}


dataset = Dataset.from_list([
    {"tokens": d["tokens"], 
     "ner_tags": [label2id[tag] for tag in d["ner_tags"]],
     "intent": intent2id[d["intent"]]}
    for d in data
])
print(dataset)

['I-COMP_OP', 'I-ACTION', 'I-TIME_FRAME', 'O', 'I-LOCATION', 'I-QUESTION_TYPE', 'I-DATE', 'B-COMP_OP', 'B-LOCATION', 'B-QUESTION_TYPE', 'B-DATE', 'I-ALERT_TYPE', 'B-TIME_FRAME', 'B-ACTION', 'B-COUNT', 'B-ALERT_TYPE', 'B-LOGIC_OP']
{'I-COMP_OP': 0, 'I-ACTION': 1, 'I-TIME_FRAME': 2, 'O': 3, 'I-LOCATION': 4, 'I-QUESTION_TYPE': 5, 'I-DATE': 6, 'B-COMP_OP': 7, 'B-LOCATION': 8, 'B-QUESTION_TYPE': 9, 'B-DATE': 10, 'I-ALERT_TYPE': 11, 'B-TIME_FRAME': 12, 'B-ACTION': 13, 'B-COUNT': 14, 'B-ALERT_TYPE': 15, 'B-LOGIC_OP': 16}
Dataset({
    features: ['tokens', 'ner_tags', 'intent'],
    num_rows: 141
})


In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [9]:


def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example["tokens"], is_split_into_words=True, truncation=True)

    labels = []
    word_ids = tokenized_inputs.word_ids()
    ner_tags = example["ner_tags"]

    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)  # Ignore special tokens
        elif word_idx != previous_word_idx:
            labels.append(ner_tags[word_idx])  # Assign NER tag
        else:
            labels.append(-100)  # Ignore subwords

        previous_word_idx = word_idx

    tokenized_inputs["labels"] = labels
    tokenized_inputs["intent_label"] = example["intent"]  # Add intent label
    return tokenized_inputs

# Apply tokenization to dataset
dataset = dataset.map(tokenize_and_align_labels)
dataset = dataset.train_test_split(test_size=0.1) 


Map:  36%|███▌      | 51/141 [00:00<00:00, 3399.81 examples/s]


ArrowInvalid: Could not convert 'B-QUESTION_TYPE' with type str: tried to convert to int64

In [105]:
from transformers import AutoTokenizer

model_checkpoint = "Davlan/xlm-roberta-base-ner-hrl"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(example["tokens"], is_split_into_words=True, truncation=True)

    labels = []
    word_ids = tokenized_inputs.word_ids()  # Maps subwords to words
    ner_tags = example["ner_tags"]  # Original labels

    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)  # Ignore special tokens
        elif word_idx != previous_word_idx:
            if word_idx < len(ner_tags):
                labels.append(label2id[ner_tags[word_idx]])  # Assign full label to first subword
            else:
                labels.append(-100)  # Handle rare out-of-bounds cases
        else:
            if word_idx < len(ner_tags):
                labels.append(label2id[ner_tags[word_idx]])  # Keep the same label
            else:
                labels.append(-100)

        previous_word_idx = word_idx

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_dataset = train_dataset.map(tokenize_and_align_labels)
test_dataset = test_dataset.map(tokenize_and_align_labels)

Map: 100%|██████████| 15/15 [00:00<00:00, 2194.52 examples/s]


# Checking if the mapping was done successfully:

In [122]:
sample = train_dataset[100]  # Get the first example
tokens = tokenizer.convert_ids_to_tokens(sample["input_ids"])  # Convert token IDs to words
labels = sample["labels"]  # Get corresponding labels

for token, label in zip(tokens, labels):
    print(f"{token:15} --> {id2label.get(label, 'IGNORED')}")


<s>             --> IGNORED
▁Has            --> B-QUESTION_TYPE
▁there          --> I-QUESTION_TYPE
▁been           --> O
▁any            --> O
▁drill          --> B-ALERT_TYPE
▁alert          --> I-ALERT_TYPE
s               --> I-ALERT_TYPE
▁this           --> B-TIME_FRAME
▁month          --> I-TIME_FRAME
▁?              --> O
</s>            --> IGNORED


In [123]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True 
)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at Davlan/xlm-roberta-base-ner-hrl and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([17]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([17, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print("Label to ID Mapping:",
            label2id)
print("ID to Label Mapping:", id2label)
print("Number of Labels:", len(label2id))

Label to ID Mapping: {'I-DATE': 0, 'I-ALERT_TYPE': 1, 'I-QUESTION_TYPE': 2, 'I-ACTION': 3, 'B-COMP_OP': 4, 'I-LOCATION': 5, 'I-COMP_OP': 6, 'O': 7, 'B-COUNT': 8, 'B-ACTION': 9, 'B-TIME_FRAME': 10, 'B-LOGIC_OP': 11, 'B-LOCATION': 12, 'B-ALERT_TYPE': 13, 'B-DATE': 14, 'B-QUESTION_TYPE': 15, 'I-TIME_FRAME': 16}
ID to Label Mapping: {0: 'I-DATE', 1: 'I-ALERT_TYPE', 2: 'I-QUESTION_TYPE', 3: 'I-ACTION', 4: 'B-COMP_OP', 5: 'I-LOCATION', 6: 'I-COMP_OP', 7: 'O', 8: 'B-COUNT', 9: 'B-ACTION', 10: 'B-TIME_FRAME', 11: 'B-LOGIC_OP', 12: 'B-LOCATION', 13: 'B-ALERT_TYPE', 14: 'B-DATE', 15: 'B-QUESTION_TYPE', 16: 'I-TIME_FRAME'}
Number of Labels: 17


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ner_bert_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,  # Adjust as needed
    weight_decay=0.01,
    push_to_hub=False,
)


In [126]:
from transformers import Trainer, DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)


In [127]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.524294
2,No log,0.990573
3,No log,0.744712
4,No log,0.642047
5,No log,0.603096


TrainOutput(global_step=80, training_loss=1.1941350936889648, metrics={'train_runtime': 172.0099, 'train_samples_per_second': 3.663, 'train_steps_per_second': 0.465, 'total_flos': 6023911197276.0, 'train_loss': 1.1941350936889648, 'epoch': 5.0})

In [132]:
!pip install evaluate

   ---------------------------------------- 0.0/84.0 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/84.0 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/84.0 kB ? eta -:--:--
   ------------------- -------------------- 41.0/84.0 kB 487.6 kB/s eta 0:00:01
   ---------------------------------------- 84.0/84.0 kB 784.8 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [133]:
import numpy as np
import evaluate  # ✅ New library for metrics

# Load the seqeval metric for NER
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [[id2label[p] for p, l in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

trainer.compute_metrics = compute_metrics
results = trainer.evaluate()
print(results)


{'eval_loss': 0.603096067905426, 'eval_precision': 0.7166666666666667, 'eval_recall': 0.6825396825396826, 'eval_f1': 0.6991869918699187, 'eval_accuracy': 0.8297872340425532, 'eval_runtime': 0.1895, 'eval_samples_per_second': 79.156, 'eval_steps_per_second': 10.554, 'epoch': 5.0}


f:\shenkar\pagmar\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [134]:
trainer.save_model("./ner_model")
tokenizer.save_pretrained("./ner_model")


('./ner_model\\tokenizer_config.json',
 './ner_model\\special_tokens_map.json',
 './ner_model\\sentencepiece.bpe.model',
 './ner_model\\added_tokens.json',
 './ner_model\\tokenizer.json')

In [13]:
from transformers import pipeline

nlp_ner = pipeline("ner", model="./ner_model", tokenizer="./ner_model", aggregation_strategy="simple")

# text = "can you tell me how many alerst were triggered in the last 24 hours?"
# text = "תראה לי את כל התראות טילים במתן."
text = "كم عدد إنذارات الصواريخ التي كانت في تل أبيب أمس؟"
result = nlp_ner(text)
print(result)


Device set to use cpu


[{'entity_group': 'QUESTION_TYPE', 'score': np.float32(0.55697435), 'word': 'كم', 'start': 0, 'end': 2}, {'entity_group': 'ALERT_TYPE', 'score': np.float32(0.5030226), 'word': 'إنذارات', 'start': 6, 'end': 14}, {'entity_group': 'ALERT_TYPE', 'score': np.float32(0.24092714), 'word': 'الص', 'start': 14, 'end': 18}, {'entity_group': 'ALERT_TYPE', 'score': np.float32(0.25940633), 'word': 'واري', 'start': 18, 'end': 22}, {'entity_group': 'ALERT_TYPE', 'score': np.float32(0.31215298), 'word': 'خ', 'start': 22, 'end': 23}, {'entity_group': 'LOCATION', 'score': np.float32(0.15332513), 'word': 'تل', 'start': 36, 'end': 39}, {'entity_group': 'LOCATION', 'score': np.float32(0.1943378), 'word': 'أبي', 'start': 39, 'end': 43}, {'entity_group': 'LOCATION', 'score': np.float32(0.21898556), 'word': 'ب', 'start': 43, 'end': 44}]
